In [ ]:
# single connect

In [7]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 11.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
import socket
import json
import time

# Server configuration
SERVER_IP = '192.168.0.241'    # or your server's IP address
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    while END_OF_MSG not in buffer:
        data = sock.recv(1024).decode('utf-8')
        if not data:
            break
        buffer += data
    # Remove the marker and return clean message.
    return buffer.replace(END_OF_MSG, "").strip()


In [23]:
# Create a TCP connection to the server and send the password
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SERVER_IP, SERVER_PORT))
    print("Connected to server.")

    # Send the shared password
    send_message(sock, SHARED_PASSWORD)
    # Wait for server response (e.g., "auth_success" or "auth_fail")
    response = receive_message(sock)
    print("Authentication response:", response)

    if response != "auth_success":
        raise Exception("Authentication failed. Check your password.")
except Exception as e:
    print("Error connecting or authenticating:", e)
    sock.close()


Connected to server.
Authentication response: auth_success


In [24]:
# Send a JSON command to request planet data
get_planets_cmd = {
    "type": "get_planets"
}
send_message(sock, json.dumps(get_planets_cmd))

# Wait and print the planet data response
planets_response = receive_message(sock)
print("Received planet data:")
print(planets_response)

# Example: Parse the JSON response (adjust parsing based on your JSON structure)
try:
    planets_data = json.loads(planets_response)
    # Assuming planets_data is a list of planets with positions; here we pick the first one.
    if isinstance(planets_data, list) and planets_data:
        selected_planet = planets_data[0]
        # Assume each planet has a "position" field with [x, y, z]
        planet_position = selected_planet.get("position", [0,0,0])
        print("Selected planet position:", planet_position)
    else:
        print("Planet data not in expected format.")
except Exception as e:
    print("Error parsing planet data:", e)


Received planet data:
{"(0, 0, 0)":[{"Position":{"x":0,"y":0,"z":0},"Seed":-1137706823,"Name":"(0, 0, 0)","ResourceLocations":[{"x":0.3737519,"y":0.21537592,"z":0.25282946},{"x":-0.038451567,"y":0.49459165,"z":0.062454604},{"x":0.0019477914,"y":0.47938702,"z":-0.14207149},{"x":0.3355466,"y":-0.27974346,"z":0.24321201},{"x":-0.032191407,"y":0.49875697,"z":-0.014324633},{"x":-0.06452342,"y":0.4557601,"z":-0.19524206},{"x":0.038518414,"y":-0.49849397,"z":-0.0044826823},{"x":-0.17041288,"y":0.027668262,"z":0.46924824},{"x":-0.0006309848,"y":0.49978003,"z":0.014816351},{"x":0.33135423,"y":0.34553376,"z":0.14425954},{"x":-0.01970671,"y":-0.4993764,"z":-0.015325747},{"x":-0.32521045,"y":0.37766024,"z":0.040135846},{"x":-0.11430149,"y":0.4730961,"z":-0.114521876},{"x":-0.13837458,"y":-0.47978696,"z":-0.025630958},{"x":-0.0877007,"y":-0.2532181,"z":0.42212456},{"x":0.3025402,"y":0.20570368,"z":-0.3408158},{"x":-0.16849813,"y":-0.11748678,"z":0.45585662},{"x":-0.01155089,"y":-0.49938002,"z":-0.0

In [25]:
import pandas as pd
from IPython.display import display

# Create a list to hold rows of data for the table.
table_data = []

# Iterate over each key-value pair in the planets_data dictionary.
# Each key is a coordinate (like "(0, 0, 0)") and the value is a list of planet dictionaries.
for coord, planets in planets_data.items():
    for planet in planets:
        # Extract position from the 'Position' dictionary.
        pos = planet.get("Position", {})
        x = pos.get("x", None)
        y = pos.get("y", None)
        z = pos.get("z", None)
        
        # Extract other details.
        name = planet.get("Name", "Unknown")
        seed = planet.get("Seed", None)
        biome = planet.get("BiomeType", None)
        
        # Append a row with the coordinate (as key) and extracted data.
        table_data.append({
            "Coordinate": coord,
            "Name": name,
            "X": x,
            "Y": y,
            "Z": z,
            "Seed": seed,
            "BiomeType": biome
        })

# Create a DataFrame and display it.
df = pd.DataFrame(table_data)
display(df)


,Coordinate,Name,X,Y,Z,Seed,BiomeType
0,"(0, 0, 0)","(0, 0, 0)",0,0,0,-1137706823,0
1,"(0, 0, 1)","(0, 0, 1)",0,0,800,-116099305,1
2,"(0, 0, 2)","(0, 0, 2)",0,0,1600,-1768693292,1
3,"(0, 1, 0)","(0, 1, 0)",0,800,0,2006072299,2
4,"(0, 1, 1)","(0, 1, 1)",0,800,800,-474678776,1
5,"(0, 1, 2)","(0, 1, 2)",0,800,1600,1219022241,2
6,"(0, 2, 0)","(0, 2, 0)",0,1600,0,149058420,1
7,"(0, 2, 1)","(0, 2, 1)",0,1600,800,-1141743011,2
8,"(0, 2, 2)","(0, 2, 2)",0,1600,1600,-242691379,1
9,"(1, 0, 0)","(1, 0, 0)",800,0,0,-1404967705,1


In [34]:
import socket
import time
import json
import threading

# Configuration (adjust these if needed)
SERVER_IP = "192.168.0.241"          # Your server's IP
SERVER_PORT = 14000                  # Your server's TCP port
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

# Utility functions
def send_message(sock, message):
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    buffer = ""
    try:
        while END_OF_MSG not in buffer:
            data = sock.recv(1024).decode('utf-8')
            if not data:
                break
            buffer += data
    except socket.timeout:
        pass  # Timeout reached; return what we have
    return buffer.replace(END_OF_MSG, "").strip()

def tcp_worker():
    try:
        # Create a new TCP connection and set a timeout
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1.0)  # 1-second timeout for recv
        sock.connect((SERVER_IP, SERVER_PORT))
        print("[Worker] Connected to server.")

        # Authenticate with the server
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print("[Worker] Authentication response:", auth_response)
        if auth_response != "auth_success":
            sock.close()
            print("[Worker] Authentication failed.")
            return

        # Define the planet's position and compute the spawn position (cube appears 5 units above the planet)
        planet_position = [0, 0, 0]
        print("[Worker] Manually set planet_position to:", planet_position)
        base_spawn = planet_position[:]  # Copy the list
        base_spawn[1] += 5.0             # Offset in Y so the cube appears above the planet
        spawn_position = base_spawn      # Use this as the spawn position

        # Build and send the spawn_cube command
        spawn_cube_cmd = {
            "type": "spawn_cube",
            "position": spawn_position,
            "rotation": [0, 0, 0]
        }
        print("[Worker] Sending spawn_cube command:", spawn_cube_cmd)
        send_message(sock, json.dumps(spawn_cube_cmd))
        time.sleep(0.1)
        spawn_resp = receive_message(sock)
        print("[Worker] Spawn cube response (if any):", spawn_resp)

        # Build and send the apply_force command
        # This command includes a "force" field; no rotation is applied.
        force_vector = [0, 100, 0]  # Adjust this force as needed
        apply_force_cmd = {
            "type": "apply_force",
            "force": force_vector
        }
        print("[Worker] Sending apply_force command:", apply_force_cmd)
        send_message(sock, json.dumps(apply_force_cmd))
        time.sleep(1)
        force_resp = receive_message(sock)
        print("[Worker] Apply force response (if any):", force_resp)

        # Close the TCP connection
        sock.close()
        print("[Worker] Connection closed.")
    except Exception as e:
        print("[Worker] Exception occurred:", e)

# Start the TCP operations in a separate thread.
worker_thread = threading.Thread(target=tcp_worker)
worker_thread.start()
worker_thread.join()
print("TCP worker thread has completed.")


[Worker] Connected to server.
[Worker] Authentication response: auth_success
[Worker] Manually set planet_position to: [0, 0, 0]
[Worker] Sending spawn_cube command: {'type': 'spawn_cube', 'position': [0, 5.0, 0], 'rotation': [0, 0, 0]}
[Worker] Spawn cube response (if any): 
[Worker] Sending apply_force command: {'type': 'apply_force', 'force': [0, 100, 0]}
[Worker] Apply force response (if any): 
[Worker] Connection closed.
TCP worker thread has completed.


In [ ]:
# Close the connection to the server gracefully.
sock.close()
print("Disconnected from server.")
